In [6]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
tqdm.pandas(position=0,leave=True)
import textwrap
import random
import time 

from transformers import AutoModelForMaskedLM , AutoTokenizer
import torch
from NLP_prompt import Prompting

import warnings
warnings.filterwarnings("ignore")

/home/ec2-user/anaconda3/envs/test/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_path="bert-large-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_path)

# load Prompting class
from prompt import Prompting
prompting= Prompting(model=model_path)

Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 52.9kB/s]
Downloading: 100%|██████████| 571/571 [00:00<00:00, 1.06MB/s]
Downloading: 100%|██████████| 226k/226k [00:00<00:00, 55.9MB/s]
Downloading: 100%|██████████| 455k/455k [00:00<00:00, 83.4MB/s]
Downloading: 100%|██████████| 1.25G/1.25G [00:11<00:00, 120MB/s]
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


KeyError: 'prompt'

In [7]:
input_dir="s3://trident-retention-output/"
output_dir="s3://trident-retention-output/output/"

start=time.time()
df=pd.read_pickle(os.path.join(output_dir,"askunum_text_bagofword"))
end=time.time()
print("It takes {:.4f} seconds to read the data".format(end-start))

It takes 235.4136 seconds to read the data


In [8]:
df.head(2)

,ParentId,Subtype,TextBody,bag_of_word,negative_word_counts
0,5000c00001TWN6bAAH,employee coding,"unum, the following associates have been termi...",following associates terminated company listed...,6
1,5000c00001TWN9pAAH,tax question,who pays the futa and sui taxes? do you odo we...,pays futa sui taxes odo report pay collect pay...,14


In [9]:
wrapper = textwrap.TextWrapper(width=150) 
# Randomly choose some examples.
for i in range(10):
    random.seed(101+i)

    j = random.choice(df.index)
    emails=df.loc[j,"TextBody"]
    subtype=df.loc[j,"Subtype"]

    print('')
    print("*"*80)
    print(f'*  Full TextBody :   subtype={subtype} *')
    print("*"*80)
    print('')
    # print(j)
    print(wrapper.fill(emails))
    print('')
    print("*"*50)



********************************************************************************
*  Full TextBody :   subtype=additional claim information received *
********************************************************************************

please see attached focourtney vandenberg, claim numbe. thank you, jessica steele hmanagebig riveresources, llc 211 north geasuite200, west
burlington, ia 52655 main 319-753-1100 direct 319-754-2106 fax 866.497.4190 what lies behind you and what lies in front of you, pales in comparison to
what lies within you. hello jessica, . i have provided the attached information focourtney to ouclaim team. if additional information is required,
theiclaim specialist will follow up. did you know you can reach oubenefits department directly with questions, fostatus updates oto submit additional
claims documents at telephone 800-858-6843 secufax 800-447-2498 email must include claim 18403832 in the subject line. please send one email
peclaimant. benefits is unable to acce

In [6]:
prompt="It has [MASK] sentiment."
prompting.compute_tokens_prob(prompt, token_list1=["positive","neutral"], token_list2= ["negative"])

tensor([0.8754, 0.1246])

In [7]:
df.loc[0,"TextBody"]

"unum, the following associates have been terminated from the company and should not be listed as covered undegerresheimeptc usa lp. dennis bussie 256-35-1645 jarrita blackmon 258-55-8371 thomas brindamou-38-5028 mark e dicicco 432-33-7949 jay fitzgerald 195-56-9599 ulysses gray 253-53-5846 georgia keith 260-13-7587 jeffrey knoblett 259-37-2420 termed 12/22/17 dana lowe 253-35-6109 warren morrow 417-98-8010 lex price 529-13-5634 domingues roberts termed 12/18/17 parwitasingelton termed 12/14/17 these individuals should have came oveon an electronic file as they wetermed in ousystem, and some of these individuals i termed in yousystem. good morning carla, please provide warren morrow's last day worked to process termination. all otheemployee's atermed as requested. please let us know if theis anything else we can assist you with in the meantime. thank you, juan gutierrez service specialist associate unum client service cente-800-ask-unum 1-800-275-8686"

In [8]:
text=df.loc[0,"TextBody"]
prompt="It has [MASK] sentiment."
prompting.prompt_pred(text+prompt)[0:10]

[('a', tensor(10.4515)),
 ('the', tensor(9.8634)),
 ('no', tensor(9.5981)),
 ('another', tensor(8.9780)),
 ('been', tensor(8.9193)),
 ('your', tensor(8.9181)),
 ('great', tensor(8.7213)),
 ('my', tensor(8.6338)),
 ('this', tensor(8.5194)),
 ('that', tensor(8.2376))]

In [9]:
prompting.compute_tokens_prob(text+prompt, token_list1=["positive","neutral"], token_list2= ["negative"])

tensor([0.9600, 0.0400])

In [10]:
df.loc[1,"TextBody"]

'who pays the futa and sui taxes? do you odo we need to report and pay those? i know you collect and pay the fica/medicare. patty lawson hmanaget 512-380-7108 f 512-459-1459 this communication and any files oattachments transmitted with it may contain information that is confidential, privileged and exempt from disclosuundeapplicable law. it is intended solely fothe use of the individual othe entity to which it is addressed. if you anot the intended recipient, you ahereby notified that any use, dissemination, ocopying of this communication is strictly prohibited. if you have received this communication in erroplease notify the sendeat once so that we may take the appropriate action. we aworking with outax unit on the request. they will be reaching out to you soon. please let us know if theis anything else we can assist with in the meantime. thank you, jessica brunhoebeservice specialist unum client service cente-800-ask-unum 1-800-275-8686 hello team i hope you awell please assist with

In [11]:
text=df.loc[1,"TextBody"]
prompt="It has [MASK] sentiment."
prompting.compute_tokens_prob(text+prompt, token_list1=["positive","neutral"], token_list2= ["negative"])

tensor([0.9790, 0.0210])

In [15]:
df.sort_values("negative_word_counts",ascending=False,inplace=True)
df=df.reset_index(drop=True)
df.head()

,ParentId,Subtype,TextBody,bag_of_word,negative_word_counts
0,5003x00002Eecc2AAB,ach inquiry or confirmation,clorox services company purchase ordeusa00152...,clorox services company purchase ordeusa power...,158
1,5000c00001axMyiAAE,claim status,"deasiomadame, i am writing on behalf of my mot...",deasiomadame writing behalf motheevereen daley...,101
2,5000c00001cy4H8AAI,employee coding,"good afternoon, could you please make suthe fo...",good afternoon suthe following employees anot ...,92
3,5003x000024WTGOAA4,correspondence change,"pejonathans request, we want to be suangie is ...",pejonathans request want suangie listed primar...,88
4,5003x00002D7WDgAAN,claim status,louisiana department of insurance p.o. box 942...,louisiana department insurance p o box baton r...,88


In [27]:
text=df.loc[1,"TextBody"]
text

"deasiomadame, i am writing on behalf of my motheevereen daley.i would like to update unum on my mothemedical conditions while she is awaiting foand still undergoing inpatient and outpatient medical treatment and care. i am unable to send this email and documentation via unum portal. she is stll in immediate need of home health caassistance with daily living and activity during the daytime, nighttime and weekend hours including skilled therapy services fophysical and occupational therapy foseven days a week. as you aalready awashe is not able to reach the bathroom in time, oforgetting whethe bathroom is. she use signage to point the way and have nightlights and emergency lights, have grab rails that she can use while reaching the bathroom, and already have clothing that can be taken off easily. she will have fecal and urinary incontinence accident in the bathroom but won't tell me about it so i won't know until i go in theand see the mess. hebrain is foggy so theis sign of dementia. sh

In [28]:
text=df.loc[1,"TextBody"]
prompt="It has [MASK] sentiment."
prompting.compute_tokens_prob(text+prompt, token_list1=["positive","neutral"], token_list2= ["negative"])

RuntimeError: The expanded size of the tensor (1454) must match the existing size (512) at non-singleton dimension 1.  Target sizes: [1, 1454].  Tensor sizes: [1, 512]

In [33]:
text=df.loc[1,"TextBody"]
prompt="It has [MASK] sentiment."
text=text+prompt

In [34]:
model_path="bert-large-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [40]:
prompt_len=len(tokenizer(prompt,add_special_tokens=False).input_ids)

In [41]:

indexed_tokens=tokenizer(text, return_tensors="pt",max_length=512-2-prompt_len).input_ids
tokenized_text= tokenizer.convert_ids_to_tokens(indexed_tokens[0])
# take the first masked token
mask_pos=tokenized_text.index(tokenizer.mask_token)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


ValueError: '[MASK]' is not in list

In [36]:
model.eval()
with torch.no_grad():
    outputs = model(indexed_tokens)
    predictions = outputs[0]
values, indices=torch.sort(predictions[0, mask_pos],  descending=True)

RuntimeError: The expanded size of the tensor (1454) must match the existing size (512) at non-singleton dimension 1.  Target sizes: [1, 1454].  Tensor sizes: [1, 512]

In [7]:
tokenizer.model_max_length, tokenizer.num_special_tokens_to_add(False)

(512, 2)

In [28]:
def compute_max_seq_len(prompt) -> int:
    """Compute the maximum sequence length of an input sequence.

    Large input sequences need to be truncated while keeping the
    prompt template untruncated.

    Returns:
        The maximum sequence length in number of tokens.
    """

    template_len=len(tokenizer(prompt,add_special_tokens=False).input_ids)

    n_special_chars = tokenizer.num_special_tokens_to_add(False)

    max_seq_len = tokenizer.model_max_length \
        - (template_len + n_special_chars)

    assert max_seq_len > 0

    return max_seq_len

text=df.loc[1,"TextBody"]
prompt="It has [MASK] sentiment"
max_seq_len=compute_max_seq_len(prompt)
max_seq_len

506

In [29]:
indexed_tokens=tokenizer(text,max_length=max_seq_len,truncation=True,add_special_tokens=False).input_ids
index_prompt=tokenizer(prompt,add_special_tokens=False).input_ids
len(indexed_tokens), len(index_prompt)

(315, 4)

In [44]:
tokenized_text= tokenizer.convert_ids_to_tokens(indexed_tokens)
print(len(tokenized_text))
tokenized_prompt=tokenizer.convert_ids_to_tokens(index_prompt)
tokenized_text=tokenized_text+tokenized_prompt
print(len(tokenized_text))

315
319


In [45]:
" ".join(tokenized_text)

'who pays the fu ##ta and sui taxes ? do you o ##do we need to report and pay those ? i know you collect and pay the fi ##ca / medicare . patty lawson hm ##ana ##get 512 - 380 - 710 ##8 f 512 - 45 ##9 - 145 ##9 this communication and any files o ##att ##ach ##ments transmitted with it may contain information that is confidential , privileged and exempt from disc ##los ##u ##und ##ea ##pp ##lica ##ble law . it is intended solely f ##oth ##e use of the individual ot ##he entity to which it is addressed . if you an ##ot the intended recipient , you ah ##ere ##by notified that any use , dissemination , o ##co ##py ##ing of this communication is strictly prohibited . if you have received this communication in er ##rop ##lea ##se not ##ify the send ##ea ##t once so that we may take the appropriate action . we aw ##or ##king with out ##ax unit on the request . they will be reaching out to you soon . please let us know if the ##is anything else we can assist with in the meantime . thank you , 

In [46]:
tokenizer.decode(indexed_tokens)

'who pays the futa and sui taxes? do you odo we need to report and pay those? i know you collect and pay the fica / medicare. patty lawson hmanaget 512 - 380 - 7108 f 512 - 459 - 1459 this communication and any files oattachments transmitted with it may contain information that is confidential, privileged and exempt from disclosuundeapplicable law. it is intended solely fothe use of the individual othe entity to which it is addressed. if you anot the intended recipient, you ahereby notified that any use, dissemination, ocopying of this communication is strictly prohibited. if you have received this communication in erroplease notify the sendeat once so that we may take the appropriate action. we aworking with outax unit on the request. they will be reaching out to you soon. please let us know if theis anything else we can assist with in the meantime. thank you, jessica brunhoebeservice specialist unum client service cente - 800 - ask - unum 1 - 800 - 275 - 8686 hello team i hope you aw

In [40]:
tokenized_text= tokenizer.decode(indexed_tokens)
# " ".join(tokenized_text)
tokenized_text

1297

In [27]:
indexed_tokens=tokenizer(" ".join(tokenized_text), return_tensors="pt").input_ids
tokenized_text= tokenizer.convert_ids_to_tokens(indexed_tokens[0])
# take the first masked token
mask_pos=tokenized_text.index(tokenizer.mask_token)

mask_pos       

484

In [11]:
# take the first masked token
mask_pos=tokenized_text.index(tokenizer.mask_token)
mask_pos       

319

In [93]:
model.eval()
with torch.no_grad():
    outputs = model(torch.long(indexed_tokens+index_prompt))
    predictions = outputs[0]
values, indices=torch.sort(predictions[0, mask_pos],  descending=True)

TypeError: 'torch.dtype' object is not callable

In [80]:
len(tokenizer(prompt,add_special_tokens=False).input_ids)

4

In [178]:
label_words=[["positive","neutral"],["negative"]]
def init_label_words(label_words: Optional[List[List[str]]]):
    """Initialize label words.

    The entire vocabulary of the tokenizer is used as
    label words if no label words are specified.

    Args:
        label_words: List of label words to use.
    """

    if label_words is None:
        # Use full vocabulary.
        label_word_ids = [x for x in range(tokenizer.vocab_size)]
        n_classes = tokenizer.vocab_size
    else:
        n_classes = len(label_words)
        label_word_ids=[[] for _ in range(n_classes)]
        for i, label in enumerate(label_words):
            for char in label:
                tokenized_label = tokenizer(char, add_special_tokens=False)['input_ids']
                label_word_ids[i].append(tokenized_label[0])
    return label_word_ids

init_label_words(label_words)

[[3893, 8699], [4997]]

In [172]:
label_words=[["positive","neutral"],["negative"]]
n_classes = len(label_words)
label_word_ids=[[] for _ in range(n_classes)]
for i, label in enumerate(label_words):
    for char in label:
        tokenized_label = tokenizer(char, add_special_tokens=False)['input_ids']
        label_word_ids[i].append(tokenized_label[0] )
label_word_ids

[[3893, 8699], [4997]]

In [174]:
range(tokenizer.vocab_size)

range(0, 30522)

In [162]:
# label_word_ids[0]
label_word_ids=[[] for _ in range(2)]
label_word_ids[0].append(1)
label_word_ids[0].append(100)
label_word_ids[1].append(2)
label_word_ids


[[1, 100], [2]]

In [154]:
label_word_ids

[0, 0]

In [15]:
def get_device():
    if torch.cuda.is_available():
        return "cuda:0"  # Use only one GPU.
    return "cpu"


input_texts = [
    "This was terrible service",
    "I want to terminate the policy."
]
# label_words = [["positive","neutral"], ["negative"]]
label_words = ["positive", "negative"]

cloze_template = ClozeTemplate(["email:", "this email has {} sentiment"])
template=cloze_template.template

In [20]:
myclass=ClozeWrapper(tokenizer,template,label_words)
max_seq_len=myclass.compute_max_seq_len()
max_seq_len

503

In [44]:
tokenizer.cls_token, tokenizer.sep_token, tokenizer.cls_token_id, tokenizer.sep_token_id, template

('[CLS]', '[SEP]', 101, 102, [[10373, 1024], [2023, 10373, 2038, 103, 15792]])

In [66]:
cloze_template = ClozeTemplate(["email:", "this email has {} sentiment"])
template=cloze_template.template
prompts = []
tokenized_inputs = tokenizer.batch_encode_plus(
            input_texts,
            add_special_tokens=False,
            return_attention_mask=False,
            truncation=True)['input_ids']

def init_template(template: List[str]):

    self_template = []
    for text_snippet in template:
        masked_text_snippet = text_snippet.format(
            tokenizer.mask_token)
        tokenized_text_snippet = tokenizer.encode(
            masked_text_snippet,
            add_special_tokens=False)
        self_template.append(tokenized_text_snippet)
    return self_template
      
template=init_template(template)

for inp in tokenized_inputs:
    truncated_input = inp[:max_seq_len]
    # prompt = itertools.chain([tokenizer.cls_token_id],template[0], truncated_input,template[1],[tokenizer.sep_token_id])
    prompt = itertools.chain(template[0], truncated_input,template[1])
    prompts.append(list(prompt))

# tokenized_prompts = tokenizer.batch_encode_plus(prompts, is_split_into_words=True)

In [67]:
print(prompts,end="\n")

[[10373, 1024, 2023, 2001, 6659, 2326, 2023, 10373, 2038, 103, 15792], [10373, 1024, 1045, 2215, 2000, 20320, 1996, 3343, 1012, 2023, 10373, 2038, 103, 15792]]


In [68]:
mask_id=[x.index(tokenizer.mask_token_id) for x in prompts]
mask_id

[9, 12]

In [69]:
[tokenizer.decode(x) for x in prompts]

['email : this was terrible service this email has [MASK] sentiment',
 'email : i want to terminate the policy. this email has [MASK] sentiment']

In [81]:
tokenized_prompts = tokenizer([tokenizer.decode(x) for x in prompts],truncation=True)
tokenized_prompts

{'input_ids': [[101, 10373, 1024, 2023, 2001, 6659, 2326, 2023, 10373, 2038, 103, 15792, 102], [101, 10373, 1024, 1045, 2215, 2000, 20320, 1996, 3343, 1012, 2023, 10373, 2038, 103, 15792, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [84]:
tokenized_prompts = tokenizer([tokenizer.decode(x) for x in prompts])
tokenized_prompts

{'input_ids': [[101, 10373, 1024, 2023, 2001, 6659, 2326, 2023, 10373, 2038, 103, 15792, 102], [101, 10373, 1024, 1045, 2215, 2000, 20320, 1996, 3343, 1012, 2023, 10373, 2038, 103, 15792, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [120]:
max_seq_len=myclass.compute_max_seq_len()
template=cloze_template.template

self_template = []
for text_snippet in template:
    masked_text_snippet = text_snippet.format(tokenizer.mask_token)
    tokenized_text_snippet = tokenizer.encode(masked_text_snippet,
                                              add_special_tokens=False)
    self_template.append(tokenized_text_snippet)

            

def gen_prompts(inputs: List[List[int]], *args,**kwargs) -> BatchEncoding:
    """Generate prompts.

    Args:
        inputs: Batch of inputs represented as lists of token ids.

    Returns:
        Prompts generated from inputs (as lists of token ids).
    """

    prompts = []
    for inp in inputs:
        truncated_input = inp[:max_seq_len]

        prompt = itertools.chain(self_template[0], truncated_input,self_template[1])

        prompts.append(list(prompt))

    # tokenized_prompts = tokenizer.batch_encode_plus(
    #     prompts, is_split_into_words=True, *args, **kwargs)

    return prompts
    
def preprocess_inputs(inputs: Union[List[str], List[int]], *args, **kwargs) -> BatchEncoding:
    """Preprocesses inputs by mapping them to their prompts.

    Args:
        inputs: List of input strings or lists of token ids.

    Returns:
        Prompts represented as lists of token ids.
    """

    is_not_pretokenized = isinstance(inputs[0], str)

    if is_not_pretokenized:
        tokenized_inputs = tokenizer.batch_encode_plus(
            inputs,
            add_special_tokens=False,
            return_attention_mask=False,
            truncation=True)['input_ids']
    else:
        tokenized_inputs = inputs

    prompts = gen_prompts(tokenized_inputs, *args, **kwargs)

    return prompts

preprocess_inputs(input_texts)

[[10373, 1024, 2023, 2001, 6659, 2326, 2023, 10373, 2038, 103, 15792],
 [10373,
  1024,
  1045,
  2215,
  2000,
  20320,
  1996,
  3343,
  1012,
  2023,
  10373,
  2038,
  103,
  15792]]

In [107]:
inputs=input_texts
is_not_pretokenized = isinstance(inputs[0], str)
tokenized_inputs = tokenizer.batch_encode_plus(
    inputs,
    add_special_tokens=False,
    return_attention_mask=False,
    truncation=True)['input_ids']

In [108]:
tokenized_inputs

[[2023, 2001, 6659, 2326], [1045, 2215, 2000, 20320, 1996, 3343, 1012]]

In [115]:
prompts = []
inputs=tokenized_inputs
for inp in inputs:
    truncated_input = inp[:max_seq_len]

    prompt = itertools.chain(self_template[0], truncated_input,self_template[1])

    prompts.append(list(prompt))

In [4]:
import torch
outputs=torch.randn(3,10,100)
label_word_ids=[[2,8],[14]]
masked_idx=8
softmax_output=True
logit=torch.zeros(outputs.shape[0],len(label_word_ids),dtype=torch.float)
for i,label_ids in enumerate(label_word_ids):
    x=0
    for ids in label_ids:
        x+=outputs[:, :, ids][:,masked_idx]
    logit[:,i]=x
if softmax_output:
    logit=logit.softmax(dim=-1)
logit.shape

torch.Size([3, 2])

In [5]:
logit

tensor([[0.5923, 0.4077],
        [0.6991, 0.3009],
        [0.9214, 0.0786]])

In [111]:
    def parse_outputs(self,
                      outputs: torch.Tensor,
                      masked_idx: torch.Tensor,
                      softmax_output=True) -> torch.Tensor:
        """Parse outputs of a MLM model.

        The MLM model outputs a probability distribution over the entire
        vocabulary for each element in an input sequence.
        This method convert the outputs associated with an
        input text to an output of the text classification problem.

        Args:
            outputs: A batch of outputs produced by the MLM model.
            masked_idx: Indices of masked positions for each output.
            softmax_output: True if softmax should be computed over the outputs.

        Returns:
            Tensor with output predictions over the classes of interest.
        """
        logit=torch.zeros(outputs.shape[0],len(label_word_ids),dtype=torch.float)
        for i,label_ids in enumerate(label_word_ids):
            x=0
            for ids in label_ids:
                x+=outputs[:, :, ids][:,masked_idx]
            logit[:,i]=x
        if softmax_output:
            logit=logit.softmax(dim=-1)

        return logits

[[10373, 1024], [2023, 10373, 2038, 103, 15792]]

In [19]:
from abc import abstractmethod
from typing import Optional, Union,List,Dict
import itertools

import torch
from transformers import PreTrainedTokenizer, PreTrainedModel
from transformers.tokenization_utils_base import BatchEncoding

"""
Classes for zero/few-shot text classification, with HuggingFace models.

Two methods are implemented:
- 'Cloze' based on Shick et. al. (2020)

Example usage:

    input_texts = [
        "This was terrible service",
        "I want to terminate the policy."
    ]
    label_words = [["positive","neutral"], ["negative"]]

    cloze_template = ClozeTemplate(["email:", "this email has {} sentiment"])
    cloze_wrapper = ClozeWrapper(
        mlm_model_tokenizer,
        cloze_template,
        label_words
    )
    cloze_preds = cloze_wrapper(mlm_model, input_texts)

"""

class ClozeTemplate():
    """Prompt template for the cloze method.

    A prompt template is given on the form [prepended_str, appended_str],
    where either prepended_str or appended_str contains a single
    empty placeholder {} representing the masked position.
    The resulting prompt of an input x is:
    prompt(x) = prepended_str + x + appended_str

    Example:
        The prompt template ["", "This text is about {}."]
        produces the prompt "[input text] This text is about [MASK]."
    """
    def __init__(self, template: List[str]):
        assert len(template) == 2

        masked_position_token = "{}"
        n_masked_position_token = sum([
            text_snippet.count(masked_position_token)
            for text_snippet in template
        ])
        assert n_masked_position_token == 1

        self.template = template

    def __repr__(self) -> str:
        template = [s.format("_") for s in self.template]
        return ''.join(['[', template[0], '][', template[1], ']'])
    

class Wrapper:
    """Zero/few-shot classification wrapper for usage with HuggingFace.

    Uses a prompt template and a list of label words to produce
    input to the underlying language model, and to parse its outputs.

    The label words define a one-to-one mapping between the tokens of the
    language model and the classes of the classification problem.
    The prompt template is used to reformulate the classification problem
    as an instance of the NLP problem the language model has been trained on.
    """
    def __init__(self, tokenizer: PreTrainedTokenizer, template, label_words):
        self.tokenizer = tokenizer
        self.init_label_words(label_words)
        self.init_template(template)
        self.device = self.get_device()
        
    def get_device(self):
        if torch.cuda.is_available():
            return "cuda:0"  # Use only one GPU.
        return "cpu"

    @classmethod
    @abstractmethod
    def init_template(cls, template):
        """Initialize prompt template."""

    @classmethod
    @abstractmethod
    def init_label_words(cls, label_words):
        """Initialize label words."""

    @classmethod
    @abstractmethod
    def preprocess_inputs(cls, self, inputs, *args, **kwargs):
        """Preprocess a batch of text inputs.

        Useful for data loader pipelines.
        """

    @classmethod
    @abstractmethod
    def forward(cls, model, inputs, softmax_output):
        """Forward pass using inputs produced by preprocess_inputs."""

    @classmethod
    @abstractmethod
    def __call__(cls, model, inputs):
        """Compute output probabilities of a batch of input texts."""

    def to_device(self, inputs):
        """Put inputs on device. """
        return {
            name: tensor.to(self.device)
            for name, tensor in inputs.items()
        }


class ClozeWrapper(Wrapper):
    """Wrapper for the Cloze Method (Schick et al. 2021).

    Text classification is reframed as a MLM problem.
    An input text is mapped to a prompt by a prompt template,
    which adds text containing a single [MASK] token to it.

    Reference:
    https://aclanthology.org/2021.eacl-main.20.pdf
    """
    def __init__(self,
                 tokenizer: PreTrainedTokenizer,
                 template: ClozeTemplate,
                 label_words: Optional[List[str]] = None):
        super().__init__(tokenizer, template, label_words)
        self.max_seq_len = self.compute_max_seq_len()

    def init_template(self, template: List[str]):
        """Initialize the prompt template.

        The prompt template is saved in its tokenized format.

        Args:
            template: The prompt template to use.
        """

        self.template = []
        for text_snippet in template:
            masked_text_snippet = text_snippet.format(
                self.tokenizer.mask_token)
            tokenized_text_snippet = self.tokenizer.encode(
                masked_text_snippet,
                add_special_tokens=False)
            self.template.append(tokenized_text_snippet)

    def compute_max_seq_len(self) -> int:
        """Compute the maximum sequence length of an input sequence.

        Large input sequences need to be truncated while keeping the
        prompt template untruncated.

        Returns:
            The maximum sequence length in number of tokens.
        """
        template_len = len(self.template[0]) + len(self.template[1])
        n_special_chars = self.tokenizer.num_special_tokens_to_add(False)

        max_seq_len = self.tokenizer.model_max_length \
            - (template_len + n_special_chars)

        assert max_seq_len > 0

        return max_seq_len

    def init_label_words(self, label_words: Optional[List[str]]):
        """Initialize label words.

        The entire vocabulary of the tokenizer is used as
        label words if no label words are specified.

        Args:
            label_words: List of label words to use.
        """

        if label_words is None:
            # Use full vocabulary.
            self.label_word_ids = [x for x in range(self.tokenizer.vocab_size)]
            self.n_classes = self.tokenizer.vocab_size
        else:
            self.n_classes = len(label_words)
            self.label_word_ids=[[] for _ in range(self.n_classes)]
            for i, label in enumerate(label_words):
                for char in label:
                    tokenized_label = self.tokenizer(char, add_special_tokens=False)['input_ids']
                    self.label_word_ids[i].append(tokenized_label[0])                

    def gen_prompts(self, inputs: List[List[int]], *args,
                    **kwargs) -> BatchEncoding:
        """Generate prompts.

        Args:
            inputs: Batch of inputs represented as lists of token ids.

        Returns:
            Prompts generated from inputs (as lists of token ids).
        """

        prompts = []
        for inp in inputs:
            truncated_input = inp[:self.max_seq_len]

            prompt = itertools.chain(self.template[0], truncated_input,self.template[1])
            
            prompts.append(list(prompt))

        # tokenized_prompts = self.tokenizer.batch_encode_plus(
        #     prompts, is_split_into_words=True, *args, **kwargs)
        # return tokenized_prompts
        return prompts

    def preprocess_inputs(self, inputs: Union[List[str], List[int]], *args,
                          **kwargs) -> BatchEncoding:
        """Preprocesses inputs by mapping them to their prompts.

        Args:
            inputs: List of input strings or lists of token ids.

        Returns:
            Prompts represented as lists of token ids.
        """

        is_not_pretokenized = isinstance(inputs[0], str)

        if is_not_pretokenized:
            tokenized_inputs = self.tokenizer.batch_encode_plus(
                inputs,
                add_special_tokens=False,
                return_attention_mask=False,
                truncation=True)['input_ids']
        else:
            tokenized_inputs = inputs

        prompts = self.gen_prompts(tokenized_inputs, *args, **kwargs)

        return prompts

    def parse_outputs(self,
                      outputs: torch.Tensor,
                      masked_idx: torch.Tensor,
                      softmax_output=True) -> torch.Tensor:
        """Parse outputs of a MLM model.

        The MLM model outputs a probability distribution over the entire
        vocabulary for each element in an input sequence.
        This method convert the outputs associated with an
        input text to an output of the text classification problem.

        Args:
            outputs: A batch of outputs produced by the MLM model.
            masked_idx: Indices of masked positions for each output.
            softmax_output: True if softmax should be computed over the outputs.

        Returns:
            Tensor with output predictions over the classes of interest.
        """

        logit=torch.zeros(outputs.shape[0],len(label_word_ids),dtype=torch.float)
        for i,label_ids in enumerate(label_word_ids):
            x=0
            for ids in label_ids:
                x+=outputs[:, :, ids][:,masked_idx]
            logit[:,i]=x
        if softmax_output:
            logit=logit.softmax(dim=-1)

        return logits

    def get_masked_idx(self, inputs: Dict[str, torch.Tensor]) -> torch.Tensor:
        """Get position of masked indices in inputs."""
        return torch.where(
            inputs['input_ids'] == self.tokenizer.mask_token_id)[1]

    def forward(self,
                model: PreTrainedModel,
                inputs: BatchEncoding,
                softmax_output=True) -> torch.Tensor:

        prompts = self.to_device(inputs)
        masked_idx = self.get_masked_idx(prompts)
        outputs = model(**prompts).logits
        parsed_outputs = self.parse_outputs(outputs, masked_idx,
                                            softmax_output)

        return parsed_outputs

    def __call__(self, model, inputs, softmax_output=True):
        with torch.no_grad():
            inputs = self.preprocess_inputs(inputs,
                                            return_tensors="pt",
                                            padding=True)
            outputs = self.forward(model, inputs, softmax_output)
        return outputs


In [ ]:
"""
Classes for zero/few-shot text classification, with HuggingFace models.

Two methods are implemented:
- 'Cloze' based on Shick et. al. (2020)
- 'Entailment' based on Yin et al. (2019)

Example usage:

    input_texts = [
        "This was the greatest movie I've ever seen!",
        "I hated this movie."
    ]
    label_words = ["positive", "negative"]

    cloze_template = ClozeTemplate(["Movie review:", "This review is {}."])
    cloze_wrapper = ClozeWrapper(
        mlm_model_tokenizer,
        cloze_template,
        label_words
    )
    cloze_preds = cloze_wrapper(mlm_model, input_texts)

    entailment_template = EntailmentTemplate("This review is {}.")
    entailment_wrapper = EntailmentWrapper(
        nli_model_tokenizer,
        entailment_template,
        label_words
    )
    entailment_preds = entailment_wrapper(nli_model, input_texts)
"""

from abc import abstractmethod
from typing import Optional, Union
import itertools

import torch
from transformers import PreTrainedTokenizer, PreTrainedModel
from transformers.tokenization_utils_base import BatchEncoding

from .utils import get_device




class EntailmentTemplate():
    """Prompt template for the entailment method.

    A prompt template is given as a string containing a single empty
    placeholder {}, which is to be replaced by label words.
    """
    def __init__(self, template: str):
        label_word_token = "{}"
        assert template.count(label_word_token) == 1

        self.template = template

    def __repr__(self) -> str:
        return self.template.format("_")





class EntailmentWrapper(Wrapper):
    """Wrapper for the entailment method (Yin et al. 2019).

    Text classification is reframed as a textual entailment problem,
    where the input text is given as the premise.
    A hypothesis is formulated for each label word, using a prompt template.
    Each hypothesis produces an output probability representing
    its likelihood, and the final probability distribution is obtained
    by taking the softmax over each of these probabilities.

    Reference:
    https://aclanthology.org/D19-1404.pdf
    """
    def __init__(self, tokenizer: PreTrainedTokenizer,
                 template: EntailmentTemplate, label_words):
        super().__init__(tokenizer, template.template, label_words)

    def init_label_words(self, label_words: list[str]):
        self.label_words = label_words
        self.n_classes = len(label_words)

    def init_template(self, template: str):
        """Initialize prompt template.

        Generates all hypotheses using the label words,
        and tokenizes them.

        Args:
            template: Prompt template to use.
        """

        self.hypotheses = self.tokenizer(
            [template.format(label_word) for label_word in self.label_words],
            add_special_tokens=False,
            return_attention_mask=False)['input_ids']

    def preprocess_inputs(self, inputs: list[str], *args,
                          **kwargs) -> BatchEncoding:
        """Tokenize inputs.

        Args:
            inputs: A batch of input texts.

        Returns:
            Tokenized inputs.
        """

        tokenized_inputs = self.tokenizer(inputs,
                                          add_special_tokens=False,
                                          return_attention_mask=False,
                                          return_length=True
                                          *args,
                                          **kwargs)

        return tokenized_inputs

    def parse_outputs(self,
                      outputs: torch.Tensor,
                      softmax_output=True) -> torch.Tensor:
        """Parse outputs of an NLI model.

        The NLI model outputs a probability distribution over
        three classes (non-entailment, neutral, entailment)
        for each (input_text, hypothesis) pair.
        This method convert the outputs associated with an
        input text to an output of the text classification problem.

        Args:
            outputs: A batch of outputs produced by the NLI model.
            softmax_output (optional): [description]. Defaults to True.

        Returns:
            Tensor with output predictions over the classes of interest.
        """

        # Ignore the neutral label.
        entailment_probs = outputs[:, [0, 2]]

        entailment_probs = entailment_probs.softmax(dim=1)
        entailment_probs = entailment_probs[:, 1]
        class_probs = torch.reshape(entailment_probs, (-1, self.n_classes))

        if softmax_output:
            class_probs = class_probs.softmax(dim=-1)

        return class_probs

    def gen_prompts(self, inputs: BatchEncoding) -> torch.Tensor:
        """Generate prompts.

        Args:
            inputs: Batch with tokenized input texts.

        Returns:
            Tensor with tokenized prompts.
        """

        if 'attention_mask' in inputs:
            # There should not be any padding between an input text (premise)
            # and a hypothesis.
            non_padded_inputs = [
                inp[attn == 1].tolist() for inp, attn in zip(
                    inputs['input_ids'], inputs['attention_mask'])
            ]
        else:
            non_padded_inputs = inputs['input_ids']

        prompts = []
        for inp in non_padded_inputs:
            for hypothesis in self.hypotheses:
                prompts.append([inp, hypothesis])

        tokenized_prompts = self.tokenizer.batch_encode_plus(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation="only_first",
            is_split_into_word=True)

        return tokenized_prompts

    def forward(self,
                model: PreTrainedModel,
                inputs: BatchEncoding,
                softmax_output: bool = True) -> torch.Tensor:

        prompts = self.gen_prompts(inputs)
        tokenized_prompts = self.to_device(prompts)
        outputs = model(**tokenized_prompts).logits
        parsed_outputs = self.parse_outputs(outputs, softmax_output)

        return parsed_outputs

    def __call__(self, model, inputs, softmax_output=True):
        with torch.no_grad():
            inputs = self.preprocess_inputs(inputs, return_tensors="np")
            outputs = self.forward(model, inputs, softmax_output)
        return outputs


In [10]:
from transformers import PreTrainedTokenizer, PreTrainedModel
from transformers.tokenization_utils_base import BatchEncoding
from transformers import AutoModelForMaskedLM , AutoTokenizer
from typing import Optional, Union,List,Dict
import torch

In [11]:
class ClozeTemplate():
    """Prompt template for the cloze method.

    A prompt template is given on the form [prepended_str, appended_str],
    where either prepended_str or appended_str contains a single
    empty placeholder {} representing the masked position.
    The resulting prompt of an input x is:
    prompt(x) = prepended_str + x + appended_str

    Example:
        The prompt template ["", "This text is about {}."]
        produces the prompt "[input text] This text is about [MASK]."
    """
    def __init__(self, template: List[str]):
        assert len(template) == 2

        masked_position_token = "{}"
        n_masked_position_token = sum([
            text_snippet.count(masked_position_token)
            for text_snippet in template
        ])
        assert n_masked_position_token == 1

        self.template = template

    def __repr__(self) -> str:
        template = [s.format("_") for s in self.template]
        return ''.join(['[', template[0], '][', template[1], ']'])
    
template=["email:", "This email has {} sentiment"]
myclass=ClozeTemplate(template)
myclass

[email:][This email has _ sentiment]

In [13]:
model_path="bert-large-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_path)
x = []
for text_snippet in template:
    masked_text_snippet = text_snippet.format(tokenizer.mask_token)
    tokenized_text_snippet = tokenizer.encode(masked_text_snippet, add_special_tokens=False)
    x.append(tokenized_text_snippet)
    
x

Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 18.9kB/s]
Downloading: 100%|██████████| 571/571 [00:00<00:00, 469kB/s]
Downloading: 100%|██████████| 226k/226k [00:00<00:00, 41.7MB/s]
Downloading: 100%|██████████| 455k/455k [00:00<00:00, 54.8MB/s]


[[10373, 1024], [2023, 10373, 2038, 103, 15792]]

In [14]:
tokenizer.encode(masked_text_snippet, add_special_tokens=False)

[2023, 10373, 2038, 103, 15792]

In [85]:
x=torch.randn(3,10,5)
x

tensor([[[ 1.0388, -0.1688,  0.1220,  0.0607,  0.5187],
         [-0.2241, -0.9021,  0.4021,  0.1500,  0.4735],
         [ 2.7840,  0.5231, -0.1815,  2.6400, -1.2372],
         [ 0.6888,  0.1721,  2.2258,  0.1697, -0.1007],
         [-0.2209,  0.6775, -1.1835, -0.3678, -0.1849],
         [-0.8734, -1.1900,  0.6574,  1.1225, -2.2173],
         [-0.8795, -0.0941, -1.3170,  0.1017,  1.3301],
         [ 0.4299,  0.9585,  1.6229,  0.0451,  0.3493],
         [ 1.6662,  1.8358,  1.1899,  0.9666, -0.1843],
         [ 0.2287,  1.0212,  0.3104,  0.0036,  0.5682]],

        [[-0.5780, -0.3849, -0.3517,  1.0626,  0.3280],
         [-0.8508,  0.1457, -0.0664, -0.2225,  1.3377],
         [-0.1595,  0.2485,  0.0106,  0.0487, -0.4561],
         [-0.2040, -1.3237, -0.1600,  1.7919,  1.0017],
         [ 0.7338,  1.3518, -1.0816,  0.6020,  1.0964],
         [-1.9872, -0.2697, -0.5012,  1.3736,  0.4318],
         [-1.3920,  1.3023,  0.3191, -0.8291, -0.7722],
         [-0.2470,  0.4126,  0.2044, -1.1355, 

In [89]:
torch.sort(x[:,8],descending=True)

torch.return_types.sort(
values=tensor([[ 1.8358,  1.6662,  1.1899,  0.9666, -0.1843],
        [ 1.0281,  0.5106,  0.3282, -0.1286, -0.7515],
        [ 1.6409,  0.6024,  0.3539, -0.0854, -1.5739]]),
indices=tensor([[1, 0, 2, 3, 4],
        [3, 4, 1, 0, 2],
        [2, 1, 4, 0, 3]]))

In [91]:
values,index=torch.sort(x[:,8],descending=True,dim=-1)
index.shape

torch.Size([3, 5])

In [92]:
values

tensor([[ 1.8358,  1.6662,  1.1899,  0.9666, -0.1843],
        [ 1.0281,  0.5106,  0.3282, -0.1286, -0.7515],
        [ 1.6409,  0.6024,  0.3539, -0.0854, -1.5739]])